In [ ]:
import os
import sys
import django

sys.path.append('../')  # add path to project root dir

os.environ["DJANGO_SETTINGS_MODULE"] = "flourish.settings"
from django.apps import apps as django_apps

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

django.setup()

In [ ]:
from flourish_caregiver.models import SocioDemographicData,HouseHoldDetails
from edc_constants.constants import NOT_APPLICABLE,YES,NO
import csv

participants= {'B142-040990096-5':{'B142-040990096-5-70':['3001M','3006M']},
                      'B142-040990786-1':{'B142-040990786-1-10':['2008M','2009M']},
                      'B142-040990786-1':{'B142-040990786-1-60':['3000C','3001M']}
}


for participant_id,child in participants.items():
    print("intial_dictionary",participant_id,child)
    for child_id,visits in child.items():
        print("child_id_dictionary",child_id,visits)
        for visit in visits:
            print("Visit we are at",visit)
            socio_demographics = SocioDemographicData.objects.filter(maternal_visit__subject_identifier=participant_id,
                                                                    maternal_visit__visit_code=visit)
            print("Socio list",socio_demographics)
            for socio_demographic in socio_demographics: 
                if child_id == socio_demographic.child_subject_identifier:
                    print("child_pid",socio_demographic.child_subject_identifier)
                    house_hold_details = HouseHoldDetails.objects.filter(
                        socio_demographics_data__maternal_visit__subject_identifier=participant_id)
                    socio_demographic.stay_with_child = house_hold_details[0].stay_with_child
                    socio_demographic.save()
                    print("Socio",socio_demographic.stay_with_child)

                    for houhouse_hold_detail in house_hold_details:
                        if houhouse_hold_detail.socio_demographics_data.child_subject_identifier == child_id:
                            print("House_hold",houhouse_hold_detail.socio_demographics_data.child_subject_identifier)
                            houhouse_hold_detail.delete()
                            


write script to correct stay_with_child "Yes" or "No" at 1000M visits to "N/A" (see blue rows in attachment for pids).

In [ ]:
socio_demographics = SocioDemographicData.objects.filter(maternal_visit__visit_code='1000M')

for socio_demographic in socio_demographics:
    if socio_demographic.stay_with_child == YES or socio_demographic.stay_with_child == NO:
        socio_demographic.stay_with_child = NOT_APPLICABLE
        socio_demographic.save()
        print(socio_demographic.maternal_visit.subject_identifier)

for subject 0369-6-10: stay_with_child at 2009M visit should not be "N/A" (yellow row in attachment). Please correct to "No". This is the value reported for `stay_with_caregiver` on Child sociodemographics at 2009 visit.

In [ ]:
subject_identifier = 'B142-040990369-6'
child_pid = 'B142-040990369-6-10'

socio_demographics = SocioDemographicData.objects.filter(maternal_visit__subject_identifier=subject_identifier,maternal_visit__visit_code='2009M')

for socio_demographic in socio_demographics:
    if socio_demographic.child_subject_identifier == child_pid:
        socio_demographic.stay_with_child = NO
        socio_demographic.save()
        print(socio_demographic.maternal_visit.subject_identifier)
    

stay_with_child and stay_with_caregiver values reported at any visit other than 1000M cannot == "N/A". must have value of "Yes" or "No"

In [ ]:
socio_demographics = SocioDemographicData.objects.exclude(maternal_visit__visit_code='1000M')

with open('participants_not_applicable.csv', mode='w', newline='') as file:
    writer = csv.writer(file)

    writer.writerow(['maternal_identifier','child_identifier', 'Visit Code','stay_with_child']) 

    for socio_demographic in socio_demographics:
        if socio_demographic.stay_with_child == NOT_APPLICABLE:
            maternal_identifier = socio_demographic.maternal_visit.subject_identifier
            child_identifier = socio_demographic.child_subject_identifier
            visit_code = socio_demographic.maternal_visit.visit_code
            stay_with_child = socio_demographic.stay_with_child

            writer.writerow([maternal_identifier,child_identifier, visit_code, stay_with_child])

        

stay_with_child in Caregiver sociodemographics and stay_with_caregiver in Child sociodemographics must have the same value across parent and child forms collected for the same child associated with the same visit code

In [ ]:
from flourish_child.models import ChildSocioDemographic

maternal_socio_demographics = SocioDemographicData.objects.all()

child_socio_demographics = ChildSocioDemographic.objects.all()

with open('participants_not_matching.csv', mode='w', newline='') as file:
    writer = csv.writer(file)

    writer.writerow(['maternal_identifier','child_identifier', 'Visit Code','stay_with_child','stay_with_caregiver']) 

    for maternal_socio_demographic in socio_demographics:
        for child_socio_demographic in child_socio_demographics:
            if child_socio_demographic.child_visit.subject_identifier == maternal_socio_demographic.child_subject_identifier and maternal_socio_demographic.maternal_visit.visit_code[:4] == child_socio_demographic.child_visit.visit_code :
                if maternal_socio_demographic.stay_with_child != child_socio_demographic.stay_with_caregiver:
                    maternal_identifier = maternal_socio_demographic.maternal_visit.subject_identifier
                    child_identifier = socio_demographic.child_subject_identifier
                    visit_code = maternal_socio_demographic.maternal_visit.visit_code
                    stay_with_child = maternal_socio_demographic.stay_with_child
                    stay_with_caregiver = child_socio_demographic.stay_with_caregiver

                    writer.writerow([maternal_identifier,child_identifier, visit_code, stay_with_child, stay_with_caregiver])